# Import Dependencies

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam
import os
import requests
import json
api_key = os.getenv("API_KEY")

# Get data

In [ ]:
url = f"https://api.oikolab.com/weather"
response = requests.get(url,
    params={'param': 'temperature',
            'location': 'Toronto, Ontario',
            'start': '1990-01-01',
            'end': '2020-12-31'},
    headers={'api-key': api_key}
    )


In [ ]:
if response.status_code == 200:
    data = response.json()
    print('Success:', response.text)
else:
    print("Error:", response.status_code, response.text)

## Save Data

In [43]:
with open('data/json/data.json', 'w') as f:
    json.dump(data, f)

## Load Data

In [ ]:
with open('data/json/data.json', 'r') as f:
    js = json.load(f)
    


In [ ]:
data = json.loads(js['data'])


df = pd.DataFrame(index=pd.to_datetime(data['index'], unit='s'),
                  data=data['data'],
                  columns=data['columns'])

print(df.head())



                       coordinates (lat,lon) model (name)  \
1990-01-01 00:00:00  (43.653482, -79.383935)         era5   
1990-01-01 01:00:00  (43.653482, -79.383935)         era5   
1990-01-01 02:00:00  (43.653482, -79.383935)         era5   
1990-01-01 03:00:00  (43.653482, -79.383935)         era5   
1990-01-01 04:00:00  (43.653482, -79.383935)         era5   

                     model elevation (surface)  utc_offset (hrs)  \
1990-01-01 00:00:00                     127.19              -5.0   
1990-01-01 01:00:00                     127.19              -5.0   
1990-01-01 02:00:00                     127.19              -5.0   
1990-01-01 03:00:00                     127.19              -5.0   
1990-01-01 04:00:00                     127.19              -5.0   

                     temperature (degC)  
1990-01-01 00:00:00                2.37  
1990-01-01 01:00:00                2.07  
1990-01-01 02:00:00                1.82  
1990-01-01 03:00:00                1.13  
1990-01-01 04:

# Train Model

In [ ]:
# Convert the temperature to fahrenheit
df['temperature (degF)'] = df['temperature (degC)'] * 9/5 + 32



In [49]:
scaler = MinMaxScaler(feature_range=(0, 1))
temperature_scaled = scaler.fit_transform(df[['temperature (degF)']].values)
window_size = 24 * 7  # Number of past days to use for predicting the next day's temperature

# Create sequences
def create_sequences(temperatures, window_size):
    X, y = [], []
    for i in range(len(temperatures) - window_size):
        X.append(temperatures[i:i + window_size])
        y.append(temperatures[i + window_size])
    return np.array(X), np.array(y).reshape(-1, 1)


# Use the normalized temperature data for creating sequences
X, y = create_sequences(temperature_scaled, window_size)

# Split the data into training and testing sets
# It's important not to shuffle time series data to maintain the temporal sequence
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

# Model Creation

In [50]:
model = Sequential([
    LSTM(100, activation='relu', input_shape=(X.shape[1], X.shape[2])),  # Increased complexity
    Dense(1)
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)

model.compile(optimizer=optimizer, loss='mean_squared_error'); 

In [51]:
model.fit(X_train, y_train, epochs=10, validation_split=0.2)

Epoch 1/10
5432/5432 [==============================] - 272s 50ms/step - loss: 0.0014 - val_loss: 1.9144e-04
Epoch 2/10
5432/5432 [==============================] - 271s 50ms/step - loss: 1.3371e-04 - val_loss: 1.1759e-04
Epoch 3/10
5432/5432 [==============================] - 272s 50ms/step - loss: 1.0827e-04 - val_loss: 1.2125e-04
Epoch 4/10
5432/5432 [==============================] - 273s 50ms/step - loss: 9.9905e-05 - val_loss: 9.1131e-05
Epoch 5/10
5432/5432 [==============================] - 274s 51ms/step - loss: 9.5974e-05 - val_loss: 1.0010e-04
Epoch 6/10
5432/5432 [==============================] - 273s 50ms/step - loss: 9.4169e-05 - val_loss: 9.0243e-05
Epoch 7/10
5432/5432 [==============================] - 273s 50ms/step - loss: 9.2582e-05 - val_loss: 8.7425e-05
Epoch 8/10
5432/5432 [==============================] - 272s 50ms/step - loss: 9.1420e-05 - val_loss: 1.0143e-04
Epoch 9/10
5432/5432 [==============================] - 271s 50ms/step - loss: 9.0391e-05 - val_loss

# Model Prediction

In [52]:
latest_data = scaler.transform(df[['temperature (degF)']].values)[-window_size:].reshape((1, window_size, 1))
predicted_temperatures_normalized = []

# Recursive prediction
for _ in range(window_size):
    # Predict the next step
    next_step_normalized = model.predict(latest_data)
    
    # Store the normalized prediction
    predicted_temperatures_normalized.append(next_step_normalized[0, 0])
    
    # Update the input sequence with the new prediction
    # This moves the window one step forward by inserting the predicted value
    latest_data = np.roll(latest_data, -1, axis=1)
    latest_data[0, -1, 0] = next_step_normalized[0, 0]

# Convert normalized predictions back to the original scale (degrees Fahrenheit)
predicted_temperatures = scaler.inverse_transform(np.array(predicted_temperatures_normalized).reshape(-1, 1)).flatten()

print("Predicted temperatures for the next 7 days (168 hours):")
print(predicted_temperatures)

1/1 [==============================] - 0s 13ms/step
Predicted temperatures for the next 7 days (168 hours):
[33.905483 33.465523 32.93237  32.36006  31.856512 31.449577 31.093943
 30.74793  30.38072  29.983288 29.567554 29.189505 28.889853 28.71249
 28.699726 29.057108 29.613482 30.218462 30.660345 31.048437 31.342964
 31.511099 31.55764  31.520348 31.632812 31.750515 31.692804 31.485434
 31.19603  30.9337   30.688753 30.428375 30.124617 29.769613 29.36868
 28.976622 28.639866 28.408974 28.334661 28.62235  29.17342  29.870655
 30.356354 30.791862 31.151985 31.40695  31.543188 31.5776   31.644724
 31.894737 31.999704 31.925186 31.72284  31.478962 31.256958 31.029312
 30.77394  30.47526  30.123055 29.729126 29.365519 29.083199 28.940039
 28.995169 29.43771  30.053774 30.585695 31.016705 31.395296 31.695274
 31.895023 31.993677 32.00877  32.12284  32.329357 32.36595  32.251537
 32.03664  31.822996 31.60927  31.378088 31.112293 30.805655 30.456772
 30.064024 29.640675 29.280916 29.046934 2

In [53]:
model.save('data/model/h5/model.h5')

In [54]:
del model

In [55]:
model = load_model('data/model/h5/model.h5')

In [42]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 100)               40800     
                                                                 
 dense_2 (Dense)             (None, 24)                2424      
                                                                 
Total params: 43,224
Trainable params: 43,224
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# This will print out the signatures of your model
print(list(model.signatures.keys()))

# Assuming 'serving_default' is a key, you can then inspect a specific signature
serving_default = model.signatures['serving_default']
print(serving_default)